# 0.3 Modeling data

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import data_scientist_challenge___latam_airlines.utils.paths as path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from datetime import time

In [3]:
synthetic_features = path.data_processed_dir("synthetic_features.csv")

In [4]:
df_model = pd.read_csv(synthetic_features)
df_model

/var/folders/mh/4z__6zcj6mv5slhwpb8j4xq40000gn/T/ipykernel_1497/2722461864.py:1: DtypeWarning: Columns (1,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_model = pd.read_csv(synthetic_features)


,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,...,SIGLAORI,SIGLADES,Hora-I,Fecha-I-sin-hora,Hora-O,Fecha-O-sin-hora,dif_mim,atraso_15,temporada_alta,periodo_dia
0,2017-01-01 23:30:00,226,SCEL,KMIA,AAL,2017-01-01 23:33:00,226,SCEL,KMIA,AAL,...,Santiago,Miami,23:30:00,2017-01-01,23:33:00,2017-01-01,3.0,no,1,noche
1,2017-01-02 23:30:00,226,SCEL,KMIA,AAL,2017-01-02 23:39:00,226,SCEL,KMIA,AAL,...,Santiago,Miami,23:30:00,2017-01-02,23:39:00,2017-01-02,9.0,no,1,noche
2,2017-01-03 23:30:00,226,SCEL,KMIA,AAL,2017-01-03 23:39:00,226,SCEL,KMIA,AAL,...,Santiago,Miami,23:30:00,2017-01-03,23:39:00,2017-01-03,9.0,no,1,noche
3,2017-01-04 23:30:00,226,SCEL,KMIA,AAL,2017-01-04 23:33:00,226,SCEL,KMIA,AAL,...,Santiago,Miami,23:30:00,2017-01-04,23:33:00,2017-01-04,3.0,no,1,noche
4,2017-01-05 23:30:00,226,SCEL,KMIA,AAL,2017-01-05 23:28:00,226,SCEL,KMIA,AAL,...,Santiago,Miami,23:30:00,2017-01-05,23:28:00,2017-01-05,-2.0,no,1,noche
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68201,2017-12-22 14:55:00,400,SCEL,SPJC,JAT,2017-12-22 15:41:00,400.0,SCEL,SPJC,JAT,...,Santiago,Lima,14:55:00,2017-12-22,15:41:00,2017-12-22,46.0,si,1,tarde
68202,2017-12-25 14:55:00,400,SCEL,SPJC,JAT,2017-12-25 15:11:00,400.0,SCEL,SPJC,JAT,...,Santiago,Lima,14:55:00,2017-12-25,15:11:00,2017-12-25,16.0,si,1,tarde
68203,2017-12-27 14:55:00,400,SCEL,SPJC,JAT,2017-12-27 15:35:00,400.0,SCEL,SPJC,JAT,...,Santiago,Lima,14:55:00,2017-12-27,15:35:00,2017-12-27,40.0,si,1,tarde
68204,2017-12-29 14:55:00,400,SCEL,SPJC,JAT,2017-12-29 15:08:00,400.0,SCEL,SPJC,JAT,...,Santiago,Lima,14:55:00,2017-12-29,15:08:00,2017-12-29,13.0,no,1,tarde


In [5]:
df_model.drop(['Ori-I','SIGLAORI','Ori-O','Fecha-I','Fecha-O','Hora-I', 'Fecha-I-sin-hora', 'Hora-O','Fecha-O-sin-hora'], axis = 'columns',inplace=True)

In [6]:
df_model.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68206 entries, 0 to 68205
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Vlo-I           68206 non-null  object 
 1   Des-I           68206 non-null  object 
 2   Emp-I           68206 non-null  object 
 3   Vlo-O           68205 non-null  object 
 4   Des-O           68206 non-null  object 
 5   Emp-O           68206 non-null  object 
 6   DIA             68206 non-null  int64  
 7   MES             68206 non-null  int64  
 8   AÑO             68206 non-null  int64  
 9   DIANOM          68206 non-null  object 
 10  TIPOVUELO       68206 non-null  object 
 11  OPERA           68206 non-null  object 
 12  SIGLADES        68206 non-null  object 
 13  dif_mim         68206 non-null  float64
 14  atraso_15       68206 non-null  object 
 15  temporada_alta  68206 non-null  int64  
 16  periodo_dia     68206 non-null  object 
dtypes: float64(1), int64(4), object

#### Transform data categorical 

In [7]:
arr_Vlo_I = df_model['Vlo-I'].unique()
count_Vlo_I = np.arange(len(arr_Vlo_I))
dic_arr_Vlo_I = dict(zip(arr_Vlo_I, count_Vlo_I))
df_model['Vlo-I'] = df_model['Vlo-I'].map(dic_arr_Vlo_I)
df_model['Vlo-I'] = df_model['Vlo-I'].astype('category')

In [8]:
arr_Des_I = df_model['Des-I'].unique()
count_Des_I = np.arange(len(arr_Des_I))
dic_arr_Des_I = dict(zip(arr_Des_I, count_Des_I))
df_model['Des-I'] = df_model['Des-I'].map(dic_arr_Des_I)
df_model['Des-I'] = df_model['Des-I'].astype('category')

In [9]:
arr_Emp_I = df_model['Emp-I'].unique()
count_Emp_I = np.arange(len(arr_Emp_I))
dic_arr_Emp_I = dict(zip(arr_Emp_I, count_Emp_I))
df_model['Emp-I'] = df_model['Emp-I'].map(dic_arr_Emp_I)
df_model['Emp-I'] = df_model['Emp-I'].astype('category')

In [10]:
arr_Vlo_O = df_model['Vlo-O'].unique()
count_Vlo_O = np.arange(len(arr_Vlo_O))
dic_arr_Vlo_O = dict(zip(arr_Vlo_O, count_Vlo_O))
df_model['Vlo-O'] = df_model['Vlo-O'].map(dic_arr_Vlo_O)
df_model['Vlo-O'] = df_model['Vlo-O'].astype('category')

In [11]:
arr_Des_O = df_model['Des-O'].unique()
count_Des_O = np.arange(len(arr_Des_O))
dic_arr_Des_O = dict(zip(arr_Des_O, count_Des_O))
df_model['Des-O'] = df_model['Des-O'].map(dic_arr_Des_O)
df_model['Des-O'] = df_model['Des-O'].astype('category')

In [12]:
arr_Emp_O = df_model['Emp-O'].unique()
count_Emp_O = np.arange(len(arr_Emp_O))
dic_arr_Emp_O = dict(zip(arr_Emp_O, count_Emp_O))
df_model['Emp-O'] = df_model['Emp-O'].map(dic_arr_Emp_O)
df_model['Emp-O'] = df_model['Emp-O'].astype('category')

In [13]:
arr_DIANOM = df_model['DIANOM'].unique()
count_DIANOM = np.arange(len(arr_DIANOM))
dic_arr_DIANOM = dict(zip(arr_DIANOM, count_DIANOM))
df_model['DIANOM'] = df_model['DIANOM'].map(dic_arr_DIANOM)
df_model['DIANOM'] = df_model['DIANOM'].astype('category')

In [14]:
arr_TIPOVUELO = df_model['TIPOVUELO'].unique()
count_TIPOVUELO = np.arange(len(arr_TIPOVUELO))
dic_arr_TIPOVUELO = dict(zip(arr_TIPOVUELO, count_TIPOVUELO))
df_model['TIPOVUELO'] = df_model['TIPOVUELO'].map(dic_arr_TIPOVUELO)
df_model['TIPOVUELO'] = df_model['TIPOVUELO'].astype('category')

In [15]:
arr_OPERA = df_model['OPERA'].unique()
count_OPERA = np.arange(len(arr_OPERA))
dic_arr_OPERA = dict(zip(arr_OPERA, count_OPERA))
df_model['OPERA'] = df_model['OPERA'].map(dic_arr_OPERA)
df_model['OPERA'] = df_model['OPERA'].astype('category')

In [16]:
arr_SIGLADES = df_model['SIGLADES'].unique()
count_SIGLADES = np.arange(len(arr_SIGLADES))
dic_arr_SIGLADES = dict(zip(arr_SIGLADES, count_SIGLADES))
df_model['SIGLADES'] = df_model['SIGLADES'].map(dic_arr_SIGLADES)
df_model['SIGLADES'] = df_model['SIGLADES'].astype('category')

In [17]:
arr_periodo_dia = df_model['periodo_dia'].unique()
count_periodo_dia = np.arange(len(arr_periodo_dia))
dic_arr_periodo_dia = dict(zip(arr_periodo_dia, count_periodo_dia))
df_model['periodo_dia'] = df_model['periodo_dia'].map(dic_arr_periodo_dia)
df_model['periodo_dia'] = df_model['periodo_dia'].astype('category')

In [18]:
df_model['dif_mim'] = df_model['dif_mim'].astype(int)

In [19]:
df_model['DIA'] = df_model['DIA'].astype('category')

In [20]:
df_model['MES'] = df_model['MES'].astype('category')

In [21]:
df_model['AÑO'] = df_model['AÑO'].astype('category')

In [22]:
df_model['temporada_alta'] = df_model['temporada_alta'].astype('category')

In [23]:
df_model['atraso_15'] = df_model['atraso_15'].astype('category')

In [24]:
df_model['target'] = df_model['atraso_15'].map({'no':0, 'si':1})
df_model['target'] = df_model['atraso_15'].map({'no':0, 'si':1})
df_model['target'] = df_model['target'].astype('category')

In [25]:
df_model.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68206 entries, 0 to 68205
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   Vlo-I           68206 non-null  category
 1   Des-I           68206 non-null  category
 2   Emp-I           68206 non-null  category
 3   Vlo-O           68206 non-null  category
 4   Des-O           68206 non-null  category
 5   Emp-O           68206 non-null  category
 6   DIA             68206 non-null  category
 7   MES             68206 non-null  category
 8   AÑO             68206 non-null  category
 9   DIANOM          68206 non-null  category
 10  TIPOVUELO       68206 non-null  category
 11  OPERA           68206 non-null  category
 12  SIGLADES        68206 non-null  category
 13  dif_mim         68206 non-null  int64   
 14  atraso_15       68206 non-null  category
 15  temporada_alta  68206 non-null  category
 16  periodo_dia     68206 non-null  category
 17  target      

In [26]:
target_column = ['target','atraso_15']

In [27]:
from sklearn.model_selection import train_test_split

In [28]:
X_train, X_test, y_train, y_test = train_test_split(
                                        df_model.drop(target_column, axis = 'columns'),
                                        df_model['target'],
                                        train_size   = 0.8,
                                        random_state = 1234,
                                        shuffle      = True
                                    )

In [29]:
print("Partición de entrenamento")
print("-----------------------")
print(y_train.describe())

Partición de entrenamento
-----------------------
count     54564
unique        2
top           0
freq      44488
Name: target, dtype: int64


In [30]:
print("Partición de test")
print("-----------------------")
print(y_test.describe())

Partición de test
-----------------------
count     13642
unique        2
top           0
freq      11104
Name: target, dtype: int64


In [31]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_selector

In [32]:
numeric_cols = X_train.select_dtypes(include=['float64', 'int']).columns.to_list()
cat_cols = X_train.select_dtypes(include=['object', 'category']).columns.to_list()

In [33]:

# Transformaciones para las variables numéricas
numeric_transformer = Pipeline(
                        steps=[
                            ('imputer', SimpleImputer(strategy='median')),
                            ('scaler', StandardScaler())
                        ]
                      )


# Transformaciones para las variables categóricas
categorical_transformer = Pipeline(
                            steps=[
                                ('imputer', SimpleImputer(strategy='most_frequent')),
                                ('onehot', OneHotEncoder(handle_unknown='ignore'))
                            ]
                          )

preprocessor = ColumnTransformer(
                    transformers=[
                        ('numeric', numeric_transformer, numeric_cols),
                        ('cat', categorical_transformer, cat_cols)
                    ],
                    remainder='passthrough'
                )

In [34]:
X_train_prep = preprocessor.fit_transform(X_train)
X_test_prep  = preprocessor.transform(X_test)

#### Creación de modelos

In [35]:
# Regresion
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

# Clasificacion
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn import neighbors

##### Metricas de Regresion

In [36]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

#### Regresión líneal 

In [37]:
lr = LinearRegression(normalize=True,)

In [38]:
#Entreno el modelo
lr.fit(X_train, y_train)

/Users/giocrisraigodoy/anaconda3/envs/data_scientist_challenge___latam_airlines/lib/python3.10/site-packages/sklearn/linear_model/_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LinearRegression())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)


  warnings.warn(


LinearRegression(normalize=True)

In [39]:
#Realizo una predicción
Y_pred = lr.predict(X_test)

In [40]:
print('DATOS DEL MODELO REGRESIÓN LINEAL SIMPLE')
print()
print('Valor de la pendiente o coeficiente "a":')
print(lr.coef_)
print('Valor de la intersección o coeficiente "b":')
print(lr.intercept_)
print()
print('La ecuación del modelo es igual a:')
print('y = ', lr.coef_, 'x ', lr.intercept_)

DATOS DEL MODELO REGRESIÓN LINEAL SIMPLE

Valor de la pendiente o coeficiente "a":
[ 4.56078706e-05 -5.98599736e-05 -7.71310248e-03  8.04562464e-06
  7.49241362e-03 -2.33799040e-03 -2.61655358e-05  3.00864977e-03
  4.61434711e-01  1.09074906e-03 -1.97526574e-02  1.02520178e-02
 -7.59692470e-03  1.45886050e-02  1.26826870e-02 -5.71045429e-03]
Valor de la intersección o coeficiente "b":
-930.6641806391493

La ecuación del modelo es igual a:
y =  [ 4.56078706e-05 -5.98599736e-05 -7.71310248e-03  8.04562464e-06
  7.49241362e-03 -2.33799040e-03 -2.61655358e-05  3.00864977e-03
  4.61434711e-01  1.09074906e-03 -1.97526574e-02  1.02520178e-02
 -7.59692470e-03  1.45886050e-02  1.26826870e-02 -5.71045429e-03] x  -930.6641806391493


In [41]:
print('Precisión del modelo:')
print(lr.score(X_train, y_train))

Precisión del modelo:
0.5325918796661386


#### Arboles de Regresion

In [42]:
dtr = DecisionTreeRegressor(max_depth=5, random_state = 0) 

In [43]:
#Entreno el modelo
dtr.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=5, random_state=0)

In [44]:
#Realizo una predicción
Y_pred = dtr.predict(X_test)

In [45]:
print('DATOS DEL MODELO ÁRBOLES DE DECISIÓN REGRESION')
print()

print('Precisión del modelo:')
print(dtr.score(X_train, y_train))

DATOS DEL MODELO ÁRBOLES DE DECISIÓN REGRESION

Precisión del modelo:
1.0


#### Regresion Random Forest

In [46]:
rfr = RandomForestRegressor(n_estimators = 300, max_depth = 8)

In [47]:
#Entreno el modelo
rfr.fit(X_train, y_train)

RandomForestRegressor(max_depth=8, n_estimators=300)

In [48]:
#Realizo una predicción
Y_pred = rfr.predict(X_test)

In [49]:
print('DATOS DEL MODELO BOSQUES ALEATORIOS REGRESION')
print()

print('Precisión del modelo:')
print(rfr.score(X_train, y_train))

DATOS DEL MODELO BOSQUES ALEATORIOS REGRESION

Precisión del modelo:
1.0


### Clasificacion

#### Regresión Logística

In [50]:
lrc = LogisticRegression(random_state = 0)

In [51]:
#Entreno el modelo
lrc.fit(X_train, y_train)

/Users/giocrisraigodoy/anaconda3/envs/data_scientist_challenge___latam_airlines/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=0)

In [52]:
#Realizo una predicción
y_pred = lrc.predict(X_test)

In [53]:
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[11104     0]
 [    0  2538]]


In [54]:
#Calculo la precisión del modelo
precision = precision_score(y_test, y_pred)
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
1.0


In [55]:
#Calculo la exactitud del modelo

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
1.0


In [56]:
#Calculo la sensibilidad del modelo
sensibilidad = recall_score(y_test, y_pred)
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
1.0


In [57]:
#Calculo el Puntaje F1 del modelo
puntajef1 = f1_score(y_test, y_pred)
print('Puntaje F1 del modelo:')
print(puntajef1)

Puntaje F1 del modelo:
1.0


In [58]:
#Calculo la curva ROC - AUC del modelo
roc_auc = roc_auc_score(y_test, y_pred)
print('Curva ROC - AUC del modelo:')
print(roc_auc)

Curva ROC - AUC del modelo:
1.0


In [59]:
print('Precisión del modelo:', precision)
print('Exactitud del modelo:', exactitud)
print('Sensibilidad del modelo:', sensibilidad)
print('Puntaje F1 del modelo:', puntajef1)
print('Curva ROC - AUC del modelo:', roc_auc)

Precisión del modelo: 1.0
Exactitud del modelo: 1.0
Sensibilidad del modelo: 1.0
Puntaje F1 del modelo: 1.0
Curva ROC - AUC del modelo: 1.0


#### Arboles de decision

In [60]:
dtc = DecisionTreeClassifier(criterion = 'entropy', random_state = 0) 

In [61]:
#Entreno el modelo
dtc.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy', random_state=0)

In [62]:
#Realizo una predicción
y_pred = dtc.predict(X_test)

In [63]:
#Verifico la matriz de Confusión
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[11104     0]
 [    0  2538]]


In [64]:
#Calculo la precisión del modelo
precision = precision_score(y_test, y_pred)
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
1.0


#### Clasificador Random Forest

In [65]:
rfc = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)

In [66]:
#Entreno el modelo
rfc.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', n_estimators=10, random_state=0)

In [67]:
#Realizo una predicción
y_pred = rfc.predict(X_test)

In [68]:
#Verifico la matriz de Confusión
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[11104     0]
 [    0  2538]]


In [69]:
#Calculo la precisión del modelo
precision = precision_score(y_test, y_pred)
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
1.0


#### Support Vector Machines (SVM)

In [70]:
svck = SVC(kernel = 'rbf', random_state = 0)

In [71]:
#Entreno el modelo
svck.fit(X_train, y_train)

SVC(random_state=0)

In [72]:
#Realizo una predicción
y_pred = svck.predict(X_test)

In [73]:
#Verifico la matriz de Confusión
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[11104     0]
 [ 1284  1254]]


In [74]:
#Calculo la precisión del modelo
precision = precision_score(y_test, y_pred)
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
1.0


#### Naive Bayes

In [75]:
gnb = GaussianNB()

In [76]:
#Entreno el modelo
gnb.fit(X_train, y_train)

GaussianNB()

In [77]:
#Realizo una predicción
y_pred = gnb.predict(X_test)

In [78]:
#Verifico la matriz de Confusión
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[10908   196]
 [  537  2001]]


In [79]:
#Calculo la precisión del modelo
precision = precision_score(y_test, y_pred)
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
0.9107874374146564


#### KNN

In [80]:
knn = neighbors.KNeighborsClassifier(n_neighbors = 2, metric = 'minkowski', p = 2)

In [81]:
#Entreno el modelo
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=2)

In [82]:
#Realizo una predicción
y_pred = knn.predict(X_test)

In [83]:
#Verifico la matriz de Confusión
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[11074    30]
 [  324  2214]]


In [84]:
#Calculo la precisión del modelo
precision = precision_score(y_test, y_pred)
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
0.9866310160427807
